In [ ]:
### Import necessary libraries
import os
from skimage.measure import label, regionprops
from skimage.filters import threshold_otsu
from skimage.color import rgb2gray
from skimage import morphology, exposure
import cv2
import pandas as pd
import numpy as np
from skimage.morphology import opening, remove_small_objects, remove_small_holes
import matplotlib.pyplot as plt
from scipy.ndimage import distance_transform_edt
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from scipy import ndimage

# Image scale: 7.0917 pixels = 1 µm
pixel_to_um = 1 / 7.0917

# Paths to the folders
bf_folder = "C:\\Users\\Admin\\Downloads\\research algue 2024\\dataframes\\BATCH\\Crystal development screen\\batch 13 consistent screen\\2025_01_07 10X chlamy screen\\BF"
pl_folder = "C:\\Users\\Admin\\Downloads\\research algue 2024\\dataframes\\BATCH\\Crystal development screen\\batch 13 consistent screen\\2025_01_07 10X chlamy screen\\PL"
output_folder = "C:\\Users\\Admin\\Downloads\\research algue 2024\\dataframes\\BATCH\\BATCH 13\\2025_1_7 10X chlamy screen"
os.makedirs(output_folder, exist_ok=True)

# List all .tif files in the folders
bf_files = sorted([f for f in os.listdir(bf_folder) if f.endswith('.tif')])
pl_files = sorted([f for f in os.listdir(pl_folder) if f.endswith('.tif')])

# Ensure there are matching files in both folders
if len(bf_files) != len(pl_files):
    raise ValueError("Mismatch in the number of BF and PL .tif files.")

#--------------------------------------------------------------
# Batch process all matching .tif files
for bf_file, pl_file in zip(bf_files, pl_files):
    print(f"Processing: {bf_file} and {pl_file}")
    
    bf_image_path = os.path.join(bf_folder, bf_file)
    pl_image_path = os.path.join(pl_folder, pl_file)

    # Load the images
    imageA = cv2.imread(bf_image_path)
    imageB = cv2.imread(pl_image_path)

    # Ensure the images are loaded properly
    if imageA is None or imageB is None:
        print(f"Skipping {bf_file} or {pl_file}: Unable to load image.")
        continue

    # Binary mask creation for Image A
    grayA = rgb2gray(imageA)
    #NEW-----------------------------------------------------------
    grayA = exposure.equalize_adapthist(grayA)
    grayA = cv2.bilateralFilter((grayA * 255).astype(np.uint8), 9, 75, 75)
    mean_intensity = np.mean(grayA)
    P1 = np.percentile(grayA, 10)
    P2 = np.percentile(grayA, 15)
    P3 = np.percentile(grayA, 20)
    dynamic_threshold = P1 if mean_intensity < 100 else (P2 if mean_intensity < 200 else P3)
    binary_A = (grayA < dynamic_threshold).astype(np.uint8)*255

    #------------------------------------------
    #dynamic_threshold = np.percentile(grayA,20)
    #binary_A = (grayA < dynamic_threshold).astype(np.uint8) * 255
    binary_A = opening(binary_A)
    binary_A = remove_small_objects(binary_A.astype(bool), min_size=1000)
    binary_A = remove_small_holes(binary_A, area_threshold=100000)
    binary_A = morphology.dilation(binary_A, morphology.disk(3))
    binary_A = morphology.closing(binary_A, morphology.disk(3))
    binary_A = (binary_A > 0).astype(np.uint8) * 255
    
    # Find regions in binary_A and calculate their areas in µm²
    region_labels_A = label(binary_A)
    region_props_A = regionprops(region_labels_A)
    
    # Apply distance transform to the binary mask
    distance = distance_transform_edt(binary_A)

    # Find local maxima to use as markers
    local_maxi = peak_local_max(distance, indices=False, labels=binary_A, min_distance=1)

    # Label the local maxima
    markers, _ = ndimage.label(local_maxi)

    # Apply watershed
    labels_watershed = watershed(-distance, markers, mask=binary_A)

    # Visualization of watershed result
    plt.figure(figsize=(8, 8))
    plt.imshow(labels_watershed, cmap='nipy_spectral')
    plt.title('Watershed Segmentation')
    plt.axis('off')
    plt.show()
    
    # Save the segmented binary image
    segmented_image_path = os.path.join(output_folder, f"{os.path.splitext(bf_file)[0]}_Segmented.png")
    cv2.imwrite(segmented_image_path, labels_watershed)
    print(f"Saved segmented image for {bf_file} to {segmented_image_path}")

    filtered_binary_A = np.zeros_like(labels_watershed)
    #filtered_binary_A = np.zeros_like(binary_A)
    for prop in region_props_A:
        #if prop.area <= 5000 and prop.area > 2000:
        #if prop.area > 3000 and prop.area < 6000:
        if prop.area > 0:
            min_row, min_col, max_row, max_col = prop.bbox
            #filtered_binary_A[min_row:max_row, min_col:max_col] = (region_labels_A[min_row:max_row, min_col:max_col] == prop.label)
            filtered_binary_A[min_row:max_row, min_col:max_col] = (labels_watershed[min_row:max_row, min_col:max_col] == prop.label)
            
    filtered_binary_A = (filtered_binary_A > 0).astype(np.uint8) * 255  # Convert back to uint8
    
    # Create a DataFrame for the regions with their area in µm²
    region_area = pd.DataFrame({
        "Region_Label": [region.label for region in region_props_A],
        "Region_Area (pixels)": [region.area for region in region_props_A],
        "Region_Area (µm²)": [region.area * (pixel_to_um ** 2) for region in region_props_A]
    })
    
    region_area_df = region_area[region_area["Region_Area (µm²)"] > 0]

    # Calculate the total area in µm²
    total_area = region_area_df["Region_Area (µm²)"].sum()

    # Add the total area to the DataFrame
    region_area_df.loc["Total"] = ["Total", "", total_area]

    # Save the DataFrame to a CSV file
    region_area_excel_path = os.path.join(output_folder, f"{os.path.splitext(bf_file)[0]}_Region_Area_in_um2.xlsx")
    region_area_df.to_excel(region_area_excel_path, index=False)

    print(f"Saved region areas for {bf_file} to {region_area_excel_path}")

    # Process Image B (contrast enhancement and thresholding)
    grayB = rgb2gray(imageB)
    grayB = exposure.equalize_adapthist(grayB)
    grayB = cv2.bilateralFilter((grayB * 255).astype(np.uint8), 9, 75, 75)
    mean_intensity = np.mean(grayB)
    std_intensity = np.std(grayB)
    dynamic_multiplier = 3 if std_intensity < 30 else (4 if std_intensity < 70 else 6)
    dynamic_threshold = mean_intensity + dynamic_multiplier * std_intensity
    binary_B = (grayB > dynamic_threshold).astype(np.uint8)
    
    # Plot both binary_A and binary_B
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))

    # Adjust indexing for 2x2 grid of axes
    #ax[0, 0].imshow(binary_min_B, cmap='gray')
    #ax[0, 0].set_title('Min Threshold Binary')
    #ax[0, 0].axis('off')  # Hide axes

    #ax[0, 1].imshow(binary_max_B, cmap='gray')
    #ax[0, 1].set_title('Max Threshold Binary')
    #ax[0, 1].axis('off')  # Hide axes

    ax[0].imshow(filtered_binary_A, cmap='gray')
    ax[0].set_title('Binary A')
    ax[0].axis('off')  # Hide axes

    ax[1].imshow(binary_B, cmap='gray')
    ax[1].set_title('Binary B')
    ax[1].axis('off')  # Hide axes

    plt.tight_layout()
    plt.show()

    # Resize for alignment
    filtered_binary_A_resized = cv2.resize(binary_A, (2048, 2048), interpolation=cv2.INTER_AREA)
    binary_B_resized = cv2.resize(binary_B, (2048, 2048), interpolation=cv2.INTER_AREA)

    # Overlap calculation
    overlap = (np.logical_and(filtered_binary_A_resized > 0, binary_B_resized > 0)).astype(np.uint8) * 255
    
    # Calculate the total overlap area in pixels
    #overlap_area = np.sum(overlap > 0)

    # Save overlap results
    #if overlap_area < 400:
    overlap_path = os.path.join(output_folder, f"{os.path.splitext(bf_file)[0]}_Overlap.png")
    cv2.imwrite(overlap_path, overlap)

    #--------------------------------------------------------------
    # Save clustering information
    region_to_cell_mapping = []
    cell_labels = label(filtered_binary_A_resized)
    cell_props = regionprops(cell_labels)
    region_labels = label(overlap)
    region_props = regionprops(region_labels)

    for region in region_props:
        region_coords = set(tuple(coord) for coord in region.coords)
        best_match_cell = None
        max_overlap = 0
        for cell in cell_props:
            cell_coords = set(tuple(coord) for coord in cell.coords)
            overlap_area = len(region_coords & cell_coords)
            if overlap_area > max_overlap:
                max_overlap = overlap_area
                best_match_cell = cell.label
        region_to_cell_mapping.append({
            "Region_Label": region.label,
            "Associated_Cell": best_match_cell,
            "Overlap (pixels)": max_overlap,
            "Region_Area (pixels)": region.area,
            "Region_Area (µm²)": region.area * (pixel_to_um ** 2)
        })

    # Save region-to-cell mapping as CSV
    df_mapp = pd.DataFrame(region_to_cell_mapping)
    df_mapping = df_mapp[df_mapp["Region_Area (µm²)"] > 0]

    # Add additional stats to the DataFrame
    df_mapping["Associated_Cell_Count"] = df_mapping["Associated_Cell"].map(df_mapping["Associated_Cell"].value_counts())
    total_distinct_cells = df_mapping["Associated_Cell"].nunique()
    df_mapping["Total_Distinct_Cells"] = total_distinct_cells
    df_mapping.loc["Total", "Region_Area (µm²)"] = df_mapping["Region_Area (µm²)"].sum()

    # Save the updated CSV
    mapping_excel_path = os.path.join(output_folder, f"{os.path.splitext(bf_file)[0]}_Region_Cell_Mapping.xlsx")
    df_mapping.to_excel(mapping_excel_path, index=False)
    
    # Group by Associated_Cell and count regions, then calculate percentages
    cell_grouped_df = pd.DataFrame(region_to_cell_mapping)
    cell_region_count = cell_grouped_df.groupby("Associated_Cell")["Region_Label"].count().reset_index()
    cell_region_count.columns = ["Associated_Cell", "Region_Count"]
    total_region_count = cell_region_count["Region_Count"].sum()
    cell_region_count["Percentage"] = (cell_region_count["Region_Count"] / total_region_count) * 100

    # Remove columns "D" and "E" from the first sheet (final grouped DataFrame)
    final_grouped_df = cell_region_count.drop(columns=["Percentage"])
    
    # Create a new DataFrame to count how many times each Associated_Cell has the same value as Region_Label
    region_to_associated_cell_mapping = []
    for region in region_to_cell_mapping:
        region_label = region["Region_Label"]
        associated_cell = region["Associated_Cell"]
        # Check if the Associated Cell matches the Region Label
        region_to_associated_cell_mapping.append({
            "Region_Label": region_label,
            "Associated_Cell": associated_cell
        })

    # Convert to DataFrame
    region_association_df = pd.DataFrame(region_to_associated_cell_mapping)

    # Count how many times each Associated_Cell has the same value as the Region_Label
    matching_cell_count = region_association_df[region_association_df["Region_Label"] == region_association_df["Associated_Cell"]].groupby("Region_Label").size().reset_index(name="Matching_Associated_Cell_Count")

    # Merge the new data with the original cell_region_count dataframe
    final_grouped_df = pd.merge(final_grouped_df, matching_cell_count, how="left", left_on="Associated_Cell", right_on="Region_Label")

    # Define the output path with .xlsx extension instead of .csv
    grouped_xlsx_path = os.path.join(output_folder, f"{os.path.splitext(bf_file)[0]}_Grouped_by_Cell_Region_Count_with_Percentage_and_Matching.xlsx")

    # Saving to an Excel file with multiple sheets
    with pd.ExcelWriter(grouped_xlsx_path, engine='xlsxwriter') as writer:
        # Save the original DataFrame (final_grouped_df) to the first sheet
        final_grouped_df.to_excel(writer, sheet_name='Cell_Region_Count', index=False)  # Shortened sheet name
        
        # Save the grouped Region_Count DataFrame to the second sheet, including percentages
        region_count_grouped = cell_region_count.groupby('Region_Count').size().reset_index(name='Region_Count_Frequency')
        
        # Remove the total count entry
        region_count_grouped = region_count_grouped[region_count_grouped['Region_Count'] != region_count_grouped['Region_Count'].sum()]

        # Add percentages to the Region_Count Grouped sheet
        total_region_count = region_count_grouped['Region_Count_Frequency'].sum()
        region_count_grouped['Percentage'] = (region_count_grouped['Region_Count_Frequency'] / total_region_count) * 100
        
        region_count_grouped.to_excel(writer, sheet_name='Region_Count_Grouped', index=False)  # Shortened sheet name

    print(f"Saved results for {bf_file} to {grouped_xlsx_path}")

    #--------------------------------------------------------------
    # Visualization
    annotated_image = imageA.copy()
    for mapping in region_to_cell_mapping:
        region_label = mapping["Region_Label"]
        associated_cell = mapping["Associated_Cell"]
        if associated_cell:
            region = next(r for r in region_props if r.label == region_label)
            min_row, min_col, max_row, max_col = region.bbox
            cv2.rectangle(annotated_image, (min_col, min_row), (max_col, max_row), (0, 255, 0), 2)
            cv2.putText(
                annotated_image,
                f"Cell {associated_cell}",
                (min_col, min_row - 5),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.3,
                (255, 0, 0),
                1
            )
            
    # Plot both binary_A and binary_B
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))

    # Show binary_A
    ax[0].imshow(annotated_image, cmap='gray')
    ax[0].set_title('Detections')
    ax[0].axis('off')  # Hide axes

    # Show binary_B
    ax[1].imshow(overlap, cmap='gray')
    ax[1].set_title('Coincidences')
    ax[1].axis('off')  # Hide axes

    plt.tight_layout()
    plt.show()
    
    # Save annotated image
    annotated_image_path = os.path.join(output_folder, f"{os.path.splitext(bf_file)[0]}_Annotated_Image_with_Clustering.png")
    cv2.imwrite(annotated_image_path, annotated_image)

    print(f"Saved results for {bf_file} to {output_folder}")